In [ ]:
import h5py
import numpy
import matplotlib.pyplot as plt
import numpy as np 
import cv2

import torch
from torchmetrics import MetricCollection, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from scipy.io import loadmat

from torchmetrics import MetricCollection, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure



In [ ]:
def scaleMinMax(x):

    x = torch.stack([x[..., 0], x[..., 1], x[..., 2]], dim=-1)  # Select RGB channels

    # Initialize an empty list to hold the scaled channels
    scaled_channels = []
    
    # Iterate over each channel (assuming the input has shape [channels, height, width])
    for c in range(x.shape[-1]):  # Iterate over channels
        channel = x[..., c]
        
        # Min-max normalization for each channel separately
        x_min = torch.min(channel)  # Minimum value for the channel
        x_max = torch.max(channel)  # Maximum value for the channel
        scaled_channel = (channel - x_min) / (x_max - x_min)
        
        # Add the scaled channel to the list
        scaled_channels.append(scaled_channel)
    
    # Stack the scaled channels back together
    scaled_image = torch.stack(scaled_channels, dim=-1)
    return scaled_image

device="cpu"
metric_collection = MetricCollection({
        'psnr': PeakSignalNoiseRatio().to(device),
        'ssim': StructuralSimilarityIndexMeasure().to(device)})


In [ ]:
i =2

In [ ]:
f = h5py.File(str("datasets/gf2/test_gf2_multiExm1.h5"), 'r+')
sr_init = torch.tensor(f['gt'][()], dtype=torch.float32).to("cpu")[2]

data = loadmat(f"cannet/results/output_mulExm_{i}.mat")['sr']
img_init = torch.asarray(data).to("cpu").permute(2, 0, 1)


In [ ]:
img_init.shape

In [ ]:
img_init.shape

In [ ]:
cropped_sr.shape

In [ ]:
i = 2

# Define the rectangle's position and size
top_left = (13, 13)  # x, y position of top-left corner of the rectangle
width, height = 20, 20  # Width and height of the rectangle
bottom_right = (top_left[0] + width, top_left[1] + height)  # bottom-right corner of the rectangle (width=50, height=50)

train_metric = metric_collection.forward(img_init.unsqueeze(0), sr_init.unsqueeze(0))
large_metric = metric_collection.compute()
metric_collection.reset()
psnr_t = large_metric['psnr'].numpy()
ssim_t = large_metric['ssim'].numpy()

cropped_hr = img_init[:, top_left[1]:top_left[1]+height, top_left[0]:top_left[0]+width]
cropped_sr = sr_init[:, top_left[1]:top_left[1]+height, top_left[0]:top_left[0]+width]
train_metric = metric_collection.forward(cropped_hr.unsqueeze(0), cropped_sr.unsqueeze(0))
cropped_large_metric = metric_collection.compute()
metric_collection.reset()
psnr_s = cropped_large_metric['psnr'].numpy()
ssim_s = cropped_large_metric['ssim'].numpy()



print(psnr_t, "", ssim_t)
print(psnr_s, "", ssim_s)
img_with_rectangle = img_init.clone()  # Make a copy of the image to modify



sr_vis = scaleMinMax(sr_init.clone().permute(2, 1, 0)).numpy()
sr_vis_rect = sr_vis.copy()

cv2.rectangle(sr_vis_rect, top_left, bottom_right, (255, 0, 0), thickness=1)



############
cropped_sr = sr_vis[top_left[1]:top_left[1]+height, top_left[0]:top_left[0]+width]

cropped_resized = cv2.resize(cropped_sr, 
                             (cropped_sr.shape[0] * 6, cropped_sr.shape[1] * 6))

h, w = sr_vis.shape[0], sr_vis.shape[1]  # Image dimensions (height, width)
h_resized, w_resized = cropped_resized.shape[:2]

top_left_resized = (w - w_resized - 2, h - h_resized - 2)  # A little padding of 10 pixels


sr_vis_rect[top_left_resized[1]:top_left_resized[1] + h_resized, 
                   top_left_resized[0]:top_left_resized[0] + w_resized] = cropped_resized

cv2.rectangle(sr_vis_rect, 
              (top_left_resized[0] - 1, top_left_resized[1] - 1), 
              (top_left_resized[0] + w_resized + 1, top_left_resized[1] + h_resized + 1),
              (0.50, 0.50, 0.50), thickness=2)  # Gray color and thickness=2

text = f"{psnr_s:.2f}/{ssim_s:.4f}"

# Use cv2.putText to add the text on the image
font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
font_scale = 0.42
font_thickness = 1

# Calculate text size and background rectangle size
text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
text_width, text_height = text_size[0], text_size[1]

# Position for the text relative to the bottom-right rectangle
text_x = 145  # Align with the left edge of the bottom-right rectangle
text_y = 132  # Slightly above the bottom-right rectangle
text_width = 106
text_height = 30

text_x_ = 150
text_y_ = 126

# Draw a filled gray rectangle as the background for the text
cv2.rectangle(sr_vis_rect,
              (text_x, text_y - text_height),  # Top-left corner of the background rectangle
              (text_x + text_width, text_y),  # Bottom-right corner of the background rectangle
              (0.50, 0.50, 0.50),  # Gray color
              thickness=-1)  # Filled rectangle

# Overlay the text on the gray rectangle
cv2.putText(sr_vis_rect, text, (text_x_, text_y_), font, font_scale, (0,0,0), font_thickness)

title = f"CanNet\n{psnr_t:.2f}/{ssim_t:.4f}"
plt.title(title, fontsize=18)  # Add the title "LagNet"
plt.imshow(sr_vis_rect)
plt.axis('off')
plt.show()


In [ ]:
text_width

In [ ]:
cropped_resized.shape

In [ ]:
sr_vis.shape

In [ ]:


# Draw a rectangle with thickness = 2 (border only, empty inside)
cv2.rectangle(img_with_rectangle, top_left, bottom_right, (255, 0, 0), thickness=1)

cropped_img = img_with_rectangle[top_left[1]:top_left[1]+height, top_left[0]:top_left[0]+width]





plt.imshow(img_with_rectangle)
plt.axis('off')  # Remove the axis for better visualization
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filtering the data to only show values for channels 2, 4, 6, and 8
channels = [2, 4, 6, 8]
psnr_line1 = [40.523, 38.421, 37.104, 36.067]
psnr_line2 = [40.242, 38.202, 37.401, 36.467]
psnr_line3 = [39.834, 38.123, 37.524, 37.067]

# Creating the plot with the filtered data
plt.style.use('seaborn-v0_8-colorblind')  # Scientific style
plt.figure(figsize=(8, 6))

plt.plot(channels, psnr_line1, marker='o', label="0", color='blue', linestyle='-')
plt.plot(channels, psnr_line2, marker='s', label="0.5", color='green', linestyle='--')
plt.plot(channels, psnr_line3, marker='^', label="1", color='red', linestyle='-.')

# Adding labels, title, legend, and grid
plt.xlabel("Number of Spectral Channels", fontsize=14, labelpad=10)
plt.ylabel("PSNR (dB)", fontsize=14, labelpad=10)
plt.legend(title="Conv Scale", fontsize=12, title_fontsize=12, loc='best')
plt.grid(True, linestyle='--', alpha=0.5)

plt.xticks(channels)

# Adjusting the layout for better presentation
plt.tight_layout()
plt.show()
